---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re 

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [58]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    f = open('university_towns.txt')
    line = f.readline()

    statename=[]
    data=[]
    with open('university_towns.txt') as file: #用with打开比较好
         #如果不转换为list，那么对于io.TextIOWrapper来说很难对数据进行处理
        for line in file.readlines():
            data.append(line)
            #print(line)


    #找出statename的名字
    for l in data:  
        if '[edit]' in l:
            s= l.replace('[edit]','')
            s =s.replace('\n','')
            statename.append(s)
    #print(statename)
    #print(len(state))

    sindex=[] #state在整个序列的位置
    region=[]
    for l in data:
        s1= l.replace('[edit]','')
        s =s1.replace('\n','')
        if len(s) < 15:
            n = data.index(l)
            sindex.append(n)
    
    #把rigion提取出来，其中里面也有state
    #     l.split('\n')
#     ku =re.sub(' \(.*\)', "",l)
#     zku = re.sub('\[.*\]',"",ku)
#     fi = re.sub('\(.*','',zku)
#     fin = fi.replace('\n','')
        l=l.replace('[edit]','')
        fin=l.split('(')[0].strip()
        region.append(fin)
    #print(region)
    #print(len(region))

    #将state那一列构造出来
    state=[]
    for i in range(len(sindex)-1):
        for j in range(sindex[i],sindex[i+1]):
            state.append(statename[i])
    for i in range(len(sindex), len(sindex)+1):
        for j in range(sindex[i-1],len(data)):
            state.append(statename[i-1])
    #print(state)
    #print(len(state))

    t = {'State':state,'RegionName':region}

    df = pd.DataFrame(t) 
    df = df.drop(sindex, axis=0)

  
    return df
get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [6]:
# t=pd.read_csv('my_answer.csv')
# t['result_s']=t[['State','A4State']].apply(lambda x:x['State']==x['A4State'],axis=1)
# t['result_r']=t[['Region','A4RegionName']].apply(lambda x:x['Region']==x['A4RegionName'],axis=1)
# print(t['result_s'].tolist().count(False))
# print(t['result_r'].tolist().count(False))

In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp=gdp.iloc[:,4:6]
    gdp.columns=['quarter','cgdp']
    #print(gdp)
    length=len(gdp['quarter'])
    cg = gdp['cgdp'].tolist()
    recession_start=[]
 
    onward=0
    for i in range(length):
        if gdp['quarter'].tolist()[i]=='2000q1':
           onward=i
           #print(onward)
    
    for i in range(length-2):##找出开始recession的位置i
      if cg[i+1]-cg[i]<0 and i>=onward:
          if cg[i+2]-cg[i+1]<0:
            recession_start.append(gdp['quarter'].tolist()[i])
            #print(gdp['quarter'].tolist()[i])
            
    return recession_start[0]
get_recession_start()

'2008q3'

In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp=gdp.iloc[:,4:6]
    gdp.columns=['quarter','cgdp']
    length=len(gdp['quarter'])
    cg = gdp['cgdp'].tolist()
    recession_start=[]
    recession_end=[]
    sloc=[]
    
    onward=0  
    for i in range(length):  #本题考虑从2000年q1开始
        if gdp['quarter'].tolist()[i]=='2000q1':
           onward=i
           #print(onward)
    
    for i in range(length-2):
      if cg[i+1]-cg[i]<0 and cg[i+2]-cg[i+1]<0 and i>=onward :
            recession_start.append(gdp['quarter'].tolist()[i])
            sloc.append(i)
            #print(gdp['quarter'].tolist()[i])

    for i in range(len(sloc)-2): ##将连续4年都跌的年份去掉，只保留最开始的那个
      if sloc[i+1]-sloc[i]<2:
         del sloc[i+1]
         del recession_start[i+1]

    recession_end=[]
    for j in range(len(sloc)):
      for i in range(length-2):
         if i>sloc[j]:
           if cg[i+1]-cg[i]>0 and cg[i+2]-cg[i+1]:
                 recession_end.append(gdp['quarter'].tolist()[i+2])           
                 #print(gdp['quarter'].tolist()[i+2])
                 recession_end.append(gdp['quarter'].tolist()[i+2])
                 break

       
    return recession_end[-1]
get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp=gdp.iloc[:,4:6]
    gdp.columns=['quarter','cgdp']
    length=len(gdp['quarter'])
    q = gdp['quarter'].tolist()
    cg = gdp['cgdp'].tolist()


    start=0  
    for i in range(length):  
        if gdp['quarter'].tolist()[i]=='2008q3':
           start=i
           #print(start)

    end=0
    for i in range(length):  
        if gdp['quarter'].tolist()[i]=='2009q4':
           end=i
           #print(end)  

    min=999999
    lowest=0
    for i in range(length):
        if i>=start and i<=end:
            if min>= cg[i]:
                min=cg[i]
                lowest=q[i]

    #print(min)
    #print(lowest)    
    return lowest
get_recession_bottom()

'2009q2'

In [71]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
  
    df=pd.DataFrame([states])
    df =df.T
    df=df.reset_index()
    df.columns=['State','states_name']#构造multiindex可以不用Multiindex的函数，可以直接set_index,然后输入一个列表

    house=pd.read_csv('City_Zhvi_AllHomes.csv')
    new=pd.merge(house,df, on='State')
    new=new.set_index(['states_name','RegionName'])#构造multiindex可以不用Multiindex的函数，可以直接set_index,然后输入一个列表
    new_column=[]
    for i in new.columns.values:
        if '99' not in i :
            new_column.append(i)

    new=new[new_column]

    new=new.iloc[:,5:]
    new=new.reset_index()
    new=new.rename(columns={'states_name':'State'})
    new=new.set_index(['State','RegionName'])
    quarter=pd.PeriodIndex(new.columns,freq='Q') ##highlight!!!! 将月度数据变成季度数据的神器， PeriodIndex，变动频率的，还有DatetimeIndex
    q_house=new.groupby(quarter,axis=1).mean()
    return q_house
    

convert_housing_data_to_quarters()

2000Q1         2000Q2         2000Q3  \
State    RegionName                                                             
New York New York                           NaN            NaN            NaN   
         Rochester                 53566.666667   53233.333333   53200.000000   
         Yonkers                  222233.333333  230900.000000  245533.333333   
         Syracuse                  56800.000000   57866.666667   57400.000000   
         Town of Greece            95100.000000   95500.000000   96333.333333   
         Albany                    91566.666667   92100.000000   91233.333333   
         Town of Poughkeepsie     131966.666667  135433.333333  139566.666667   
         New Rochelle             362766.666667  370866.666667  382233.333333   
         Town of Colonie          110600.000000  112400.000000  113166.666667   
         Mount Vernon             239333.333333  247300.000000  258633.333333   
         Utica                     50700.000000   51233.333333   52600.000000   
         Valley Stream            223166.666667  225166.666667  229133.333333   
         Schenectady               72266.666667   73100.000000   73733.333333   
         White Plains             284333.333333  297833.333333  314366.666667   
         Town of Hempstead        170166.666667  172266.666667  176800.000000   
         Town of Binghamton        81633.333333   82500.000000   81700.000000   
         Town of Newburgh         116000.000000  118400.000000  120233.333333   
         Troy                      74733.333333   76533.333333   78866.666667   
         Niagara Falls             50300.000000   50133.333333   50433.333333   
         Irondequoit               83400.000000   83500.000000   84400.000000   
         Binghamton                51766.666667   52000.000000   53366.666667   
         Massapequa               248833.333333  251233.333333  254500.000000   
         Town of Elmira            51200.000000   51600.000000   51700.000000   
         Henrietta                 94566.666667   94400.000000   95766.666667   
         Town of Webster          126433.333333  128600.000000  130833.333333   
         Patchogue                134400.000000  135200.000000  141966.666667   
         Clifton Park             140633.333333  142500.000000  143466.666667   
         Freeport                 183833.333333  186433.333333  190400.000000   
         Lindenhurst              168733.333333  169100.000000  173033.333333   
         Levittown                189966.666667  193366.666667  197866.666667   
...                                         ...            ...            ...   
Vermont  Plainfield                         NaN            NaN            NaN   
         West Brattleboro                   NaN            NaN            NaN   
         Duxbury                            NaN            NaN            NaN   
         Rochester                          NaN            NaN            NaN   
         Sharon                             NaN            NaN            NaN   
         White River Junction               NaN            NaN            NaN   
         Irasburg                           NaN            NaN            NaN   
         Mt Holly                           NaN            NaN            NaN   
         Ferrisburgh                        NaN            NaN            NaN   
         Lincoln                            NaN            NaN            NaN   
         Franklin                           NaN            NaN            NaN   
         Bridport                           NaN            NaN            NaN   
         Fayston                            NaN            NaN            NaN   
         Tunbridge                          NaN            NaN            NaN   
         Ryegate                            NaN            NaN            NaN   
         Shaftsbury                         NaN            NaN            NaN   
         Shoreham                           NaN            NaN          

In [75]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    university_town=get_list_of_university_towns()
    hd=convert_housing_data_to_quarters()
    start=get_recession_start()
    bottom=get_recession_bottom()
    
    hd['change']=hd[start]-hd[bottom]
    hd=hd['change']

    university_price=pd.merge(university_town,hd,how='inner',on=['State','RegionName'])
    university_price=university_price.dropna()
    non=pd.merge(university_town,hd,how='outer',on=['State','RegionName'])
    non=non.dropna()


    t,p=ttest_ind(university_price['change'],non['change'] )
    different=True if p<0.01 else False
    better = "university town" if university_price['change'].mean()<non['change'].mean() else "non-university town"
    
    
    
    return (different,p,better)
run_ttest()

(True, 0.005153372930261777, 'university town')